In [12]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import optuna
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [14]:
import pandas as pd
import numpy as np
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [15]:
#Replace 0 values with NaN in specific columns
cols_with_zero = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_zero] = df[cols_with_zero].replace(0, np.nan)
df.fillna(df.mean(), inplace=True)
print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [16]:
x=df.drop('Outcome',axis=1)
y=df['Outcome'] 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)
print(f'Train shape: {x_train.shape}, Test shape: {x_test.shape}')

Train shape: (614, 8), Test shape: (154, 8)


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

#Define the objective function for Optuna
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, x_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [20]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-08-24 17:59:40,847] A new study created in memory with name: no-name-799af360-b9e2-4337-81c3-cf06acc5e5eb
[I 2025-08-24 17:59:41,137] Trial 0 finished with value: 0.7735931771082417 and parameters: {'n_estimators': 60, 'max_depth': 6}. Best is trial 0 with value: 0.7735931771082417.
[I 2025-08-24 17:59:41,441] Trial 1 finished with value: 0.7703570859238004 and parameters: {'n_estimators': 76, 'max_depth': 20}. Best is trial 0 with value: 0.7735931771082417.
[I 2025-08-24 17:59:41,826] Trial 2 finished with value: 0.7622030926191615 and parameters: {'n_estimators': 101, 'max_depth': 4}. Best is trial 0 with value: 0.7735931771082417.
[I 2025-08-24 17:59:42,135] Trial 3 finished with value: 0.7817471704128806 and parameters: {'n_estimators': 71, 'max_depth': 16}. Best is trial 3 with value: 0.7817471704128806.
[I 2025-08-24 17:59:43,009] Trial 4 finished with value: 0.7768452096285668 and parameters: {'n_estimators': 196, 'max_depth': 5}. Best is trial 3 with value: 0.7817471704

In [21]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7882751474573569
Best hyperparameters: {'n_estimators': 111, 'max_depth': 13}


In [24]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(x_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


Samplers IN Optuna

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, x_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [28]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2025-08-24 18:27:18,966] A new study created in memory with name: no-name-495d12b2-be31-4454-8369-383dd426e957
[I 2025-08-24 18:27:19,716] Trial 0 finished with value: 0.7849992029332058 and parameters: {'n_estimators': 156, 'max_depth': 13}. Best is trial 0 with value: 0.7849992029332058.
[I 2025-08-24 18:27:20,391] Trial 1 finished with value: 0.7735772357723577 and parameters: {'n_estimators': 160, 'max_depth': 5}. Best is trial 0 with value: 0.7849992029332058.
[I 2025-08-24 18:27:20,992] Trial 2 finished with value: 0.7540730113183485 and parameters: {'n_estimators': 152, 'max_depth': 4}. Best is trial 0 with value: 0.7849992029332058.
[I 2025-08-24 18:27:21,230] Trial 3 finished with value: 0.7670731707317073 and parameters: {'n_estimators': 56, 'max_depth': 11}. Best is trial 0 with value: 0.7849992029332058.
[I 2025-08-24 18:27:21,526] Trial 4 finished with value: 0.780121154152718 and parameters: {'n_estimators': 58, 'max_depth': 13}. Best is trial 0 with value: 0.784999202

In [29]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7866411605292524
Best hyperparameters: {'n_estimators': 119, 'max_depth': 13}


In [31]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(x_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.77


gridsearch

In [32]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [33]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-08-24 18:28:26,408] A new study created in memory with name: no-name-e8762efe-a051-4e3c-8055-335c4234f5df
[I 2025-08-24 18:28:26,839] Trial 0 finished with value: 0.7654391838036028 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7654391838036028.
[I 2025-08-24 18:28:27,637] Trial 1 finished with value: 0.7735772357723577 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 1 with value: 0.7735772357723577.
[I 2025-08-24 18:28:27,893] Trial 2 finished with value: 0.7687151283277539 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 1 with value: 0.7735772357723577.
[I 2025-08-24 18:28:28,553] Trial 3 finished with value: 0.7752351347042882 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 3 with value: 0.7752351347042882.
[I 2025-08-24 18:28:29,170] Trial 4 finished with value: 0.7703491152558585 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 3 with value: 0.775235

In [34]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7817391997449387
Best hyperparameters: {'n_estimators': 50, 'max_depth': 10}


In [36]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(x_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


optuna visualization

In [37]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [38]:
# 1. Optimization History
plot_optimization_history(study).show()

In [39]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [ ]:
# 3. Contour Plot
plot_contour(study).show()

In [ ]:
# 4. Hyperparameter Importance
plot_param_importances(study).show()

optimizing multiple ml models

In [43]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [47]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, x_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [48]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-08-24 18:32:08,462] A new study created in memory with name: no-name-297d4e19-4d0a-4973-8e2f-bfcbee674374
[I 2025-08-24 18:32:08,684] Trial 0 finished with value: 0.7686991869918699 and parameters: {'classifier': 'SVM', 'C': 36.394257072006, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 0 with value: 0.7686991869918699.
[I 2025-08-24 18:32:08,735] Trial 1 finished with value: 0.7686991869918699 and parameters: {'classifier': 'SVM', 'C': 4.815697033385626, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 0 with value: 0.7686991869918699.
[I 2025-08-24 18:32:08,783] Trial 2 finished with value: 0.7686991869918699 and parameters: {'classifier': 'SVM', 'C': 6.330134495809497, 'kernel': 'linear', 'gamma': 'scale'}. Best is trial 0 with value: 0.7686991869918699.
[I 2025-08-24 18:32:08,804] Trial 3 finished with value: 0.7003427387215049 and parameters: {'classifier': 'SVM', 'C': 4.645930654182819, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.7686991

In [49]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'RandomForest', 'n_estimators': 72, 'max_depth': 18, 'min_samples_split': 10, 'min_samples_leaf': 6, 'bootstrap': True}
Best trial accuracy: 0.7817471704128806


In [50]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.768699,2025-08-24 18:32:08.463660,2025-08-24 18:32:08.684540,0 days 00:00:00.220880,36.394257,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.768699,2025-08-24 18:32:08.685615,2025-08-24 18:32:08.735220,0 days 00:00:00.049605,4.815697,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
2,2,0.768699,2025-08-24 18:32:08.735220,2025-08-24 18:32:08.783111,0 days 00:00:00.047891,6.330134,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.700343,2025-08-24 18:32:08.784110,2025-08-24 18:32:08.804794,0 days 00:00:00.020684,4.645931,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
4,4,0.744301,2025-08-24 18:32:08.805924,2025-08-24 18:32:10.128831,0 days 00:00:01.322907,NaN,NaN,GradientBoosting,NaN,NaN,0.062959,16.0,9.0,2.0,115.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.776861,2025-08-24 18:32:58.144343,2025-08-24 18:32:58.455707,0 days 00:00:00.311364,NaN,True,RandomForest,NaN,NaN,NaN,19.0,6.0,10.0,50.0,COMPLETE
96,96,0.765455,2025-08-24 18:32:58.458705,2025-08-24 18:32:58.985702,0 days 00:00:00.526997,NaN,False,RandomForest,NaN,NaN,NaN,19.0,1.0,10.0,123.0,COMPLETE
97,97,0.770325,2025-08-24 18:32:58.986703,2025-08-24 18:32:59.471787,0 days 00:00:00.485084,NaN,True,RandomForest,NaN,NaN,NaN,8.0,2.0,10.0,66.0,COMPLETE
98,98,0.768715,2025-08-24 18:32:59.471787,2025-08-24 18:32:59.864010,0 days 00:00:00.392223,NaN,True,RandomForest,NaN,NaN,NaN,18.0,1.0,9.0,81.0,COMPLETE


In [51]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

params_classifier
GradientBoosting    0.754611
RandomForest        0.771591
SVM                 0.732892
Name: value, dtype: float64

In [52]:
# 1. Optimization History
plot_optimization_history(study).show()

In [53]:
# 3. Slice Plot
plot_slice(study).show()

In [54]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [58]:
! pip install optuna-integration[xgboost]

Defaulting to user installation because normal site-packages is not writeable


In [59]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
x, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dtest = xgb.DMatrix(x_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2025-08-24 18:35:07,185] A new study created in memory with name: no-name-94a9d85e-2ea9-4bad-af66-346ca19c6723


[0]	train-mlogloss:1.01061	eval-mlogloss:1.00531
[1]	train-mlogloss:0.96839	eval-mlogloss:0.96009
[2]	train-mlogloss:0.90197	eval-mlogloss:0.88995
[3]	train-mlogloss:0.83435	eval-mlogloss:0.81783
[4]	train-mlogloss:0.77581	eval-mlogloss:0.75796
[5]	train-mlogloss:0.72051	eval-mlogloss:0.69932
[6]	train-mlogloss:0.67317	eval-mlogloss:0.64850
[7]	train-mlogloss:0.63092	eval-mlogloss:0.60154
[8]	train-mlogloss:0.59192	eval-mlogloss:0.55964
[9]	train-mlogloss:0.55523	eval-mlogloss:0.52111
[10]	train-mlogloss:0.52668	eval-mlogloss:0.49119
[11]	train-mlogloss:0.49649	eval-mlogloss:0.45822
[12]	train-mlogloss:0.47408	eval-mlogloss:0.43404
[13]	train-mlogloss:0.44770	eval-mlogloss:0.40588
[14]	train-mlogloss:0.42439	eval-mlogloss:0.38024
[15]	train-mlogloss:0.40721	eval-mlogloss:0.36261
[16]	train-mlogloss:0.38944	eval-mlogloss:0.34273
[17]	train-mlogloss:0.37069	eval-mlogloss:0.32252
[18]	train-mlogloss:0.35665	eval-mlogloss:0.30739
[19]	train-mlogloss:0.34210	eval-mlogloss:0.29226
[20]	train

[I 2025-08-24 18:35:07,805] Trial 0 finished with value: 1.0 and parameters: {'lambda': 0.04853029097830509, 'alpha': 0.641170607666686, 'eta': 0.07304145227657399, 'gamma': 0.033982279827573965, 'max_depth': 5, 'min_child_weight': 4, 'subsample': 0.6711163926375628, 'colsample_bytree': 0.6225440949457399}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:1.01205	eval-mlogloss:1.01300
[1]	train-mlogloss:0.93392	eval-mlogloss:0.92837
[2]	train-mlogloss:0.86527	eval-mlogloss:0.85668
[3]	train-mlogloss:0.79940	eval-mlogloss:0.78919
[4]	train-mlogloss:0.74422	eval-mlogloss:0.73238
[5]	train-mlogloss:0.69034	eval-mlogloss:0.67429
[6]	train-mlogloss:0.64378	eval-mlogloss:0.62292
[7]	train-mlogloss:0.60325	eval-mlogloss:0.57851
[8]	train-mlogloss:0.56492	eval-mlogloss:0.53680
[9]	train-mlogloss:0.53004	eval-mlogloss:0.49969
[10]	train-mlogloss:0.49725	eval-mlogloss:0.46548
[11]	train-mlogloss:0.46680	eval-mlogloss:0.43299
[12]	train-mlogloss:0.43979	eval-mlogloss:0.40433
[13]	train-mlogloss:0.41345	eval-mlogloss:0.37711
[14]	train-mlogloss:0.39032	eval-mlogloss:0.35130
[15]	train-mlogloss:0.36837	eval-mlogloss:0.32750
[16]	train-mlogloss:0.34758	eval-mlogloss:0.30513
[17]	train-mlogloss:0.32927	eval-mlogloss:0.28526
[18]	train-mlogloss:0.31268	eval-mlogloss:0.26776
[19]	train-mlogloss:0.29637	eval-mlogloss:0.25085
[20]	train

[I 2025-08-24 18:35:08,587] Trial 1 finished with value: 1.0 and parameters: {'lambda': 1.4072232781275306e-08, 'alpha': 1.7907361818725914e-06, 'eta': 0.06990160787455993, 'gamma': 1.1000127725413894e-06, 'max_depth': 3, 'min_child_weight': 4, 'subsample': 0.5856487529691015, 'colsample_bytree': 0.790132099809022}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.01504	eval-mlogloss:1.01136
[1]	train-mlogloss:0.97766	eval-mlogloss:0.97087


[I 2025-08-24 18:35:12,503] Trial 48 pruned. Trial was pruned at iteration 2.


[0]	train-mlogloss:0.95053	eval-mlogloss:0.94077
[1]	train-mlogloss:0.87698	eval-mlogloss:0.86949


[I 2025-08-24 18:35:12,540] Trial 49 pruned. Trial was pruned at iteration 2.


Best trial: {'lambda': 0.04853029097830509, 'alpha': 0.641170607666686, 'eta': 0.07304145227657399, 'gamma': 0.033982279827573965, 'max_depth': 5, 'min_child_weight': 4, 'subsample': 0.6711163926375628, 'colsample_bytree': 0.6225440949457399}
Best accuracy: 1.0


In [60]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()